In [1]:
!pip install emoji


[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from tracemalloc import stop
import numpy as np
import pandas as pd
import re
import seaborn as sns
import string
import nltk
import emoji
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
import string

In [34]:
emoji_df = pd.read_csv(r"ijstable.csv")

In [35]:
emoji_df.head()

,Char,Unnamed: 1,Unicode,Occurrences,Position,Neg,Neut,Pos,Sentiment score,Sentiment bar,Unicode name,Unicode block
0,NaN,NaN,codepoint,[5...max],[0...1],[0...1],[0...1],[0...1],[-1...+1],(c.i. 95%),NaN,NaN
1,😂,NaN,0x1f602,14622,0.805,0.247,0.285,0.468,0.221,NaN,FACE WITH TEARS OF JOY,Emoticons
2,❤,NaN,0x2764,8050,0.747,0.044,0.166,0.79,0.746,NaN,HEAVY BLACK HEART,Dingbats
3,♥,NaN,0x2665,7144,0.754,0.035,0.272,0.693,0.657,NaN,BLACK HEART SUIT,Miscellaneous Symbols
4,😍,NaN,0x1f60d,6359,0.765,0.052,0.219,0.729,0.678,NaN,SMILING FACE WITH HEART-SHAPED EYES,Emoticons


In [36]:
emoji_df = emoji_df[1:]

In [37]:
emoji_df.reset_index().drop(['index'],axis=1)

,Char,Unnamed: 1,Unicode,Occurrences,Position,Neg,Neut,Pos,Sentiment score,Sentiment bar,Unicode name,Unicode block
0,😂,NaN,0x1f602,14622,0.805,0.247,0.285,0.468,0.221,NaN,FACE WITH TEARS OF JOY,Emoticons
1,❤,NaN,0x2764,8050,0.747,0.044,0.166,0.79,0.746,NaN,HEAVY BLACK HEART,Dingbats
2,♥,NaN,0x2665,7144,0.754,0.035,0.272,0.693,0.657,NaN,BLACK HEART SUIT,Miscellaneous Symbols
3,😍,NaN,0x1f60d,6359,0.765,0.052,0.219,0.729,0.678,NaN,SMILING FACE WITH HEART-SHAPED EYES,Emoticons
4,😭,NaN,0x1f62d,5526,0.803,0.436,0.22,0.343,-0.093,NaN,LOUDLY CRYING FACE,Emoticons
...,...,...,...,...,...,...,...,...,...,...,...,...
746,♮,NaN,0x266e,5,0.937,0.125,0.625,0.25,0.125,NaN,MUSIC NATURAL SIGN,Miscellaneous Symbols
747,🅾,NaN,0x1f17e,5,0.977,0.375,0.375,0.25,-0.125,NaN,NEGATIVE SQUARED LATIN CAPITAL LETTER O,Enclosed Alphanumeric Supplement
748,🔄,NaN,0x1f504,5,0.971,0.125,0.75,0.125,0,NaN,ANTICLOCKWISE DOWNWARDS AND UPWARDS OPEN CIRCL...,Miscellaneous Symbols and Pictographs
749,☄,NaN,0x2604,5,0.435,0.125,0.75,0.125,0,NaN,COMET,Miscellaneous Symbols


In [38]:
emoji_df.dropna(axis=1,inplace=True)

In [39]:
emoji_df.head()

,Char,Unicode,Occurrences,Position,Neg,Neut,Pos,Sentiment score,Unicode name,Unicode block
1,😂,0x1f602,14622,0.805,0.247,0.285,0.468,0.221,FACE WITH TEARS OF JOY,Emoticons
2,❤,0x2764,8050,0.747,0.044,0.166,0.79,0.746,HEAVY BLACK HEART,Dingbats
3,♥,0x2665,7144,0.754,0.035,0.272,0.693,0.657,BLACK HEART SUIT,Miscellaneous Symbols
4,😍,0x1f60d,6359,0.765,0.052,0.219,0.729,0.678,SMILING FACE WITH HEART-SHAPED EYES,Emoticons
5,😭,0x1f62d,5526,0.803,0.436,0.22,0.343,-0.093,LOUDLY CRYING FACE,Emoticons


In [13]:
emoji_df['Neg'].astype(float)

1      0.247
2      0.044
3      0.035
4      0.052
5      0.436
       ...  
747    0.125
748    0.375
749    0.125
750    0.125
751    0.125
Name: Neg, Length: 751, dtype: float64

In [15]:
# emoji_df['Neg'].unique()

array(['0.247', '0.044', '0.035', '0.052', '0.436', '0.053', '0.06',
       '0.094', '0.042', '0.104', '0.127', '0.062', '0.115', '0.591',
       '0.081', '0.113', '0.112', '0.1', '0.101', '0.164', '0.072',
       '0.137', '0.045', '0.043', '0.464', '0.264', '0.039', '0.006',
       '0.04', '0.033', '0.327', '0.051', '0.031', '0.09', '0.028',
       '0.532', '0.106', '0.384', '0.046', '0.135', '0.422', '0.097',
       '0.041', '0.141', '0.124', '0.281', '0.493', '0.192', '0.078',
       '0.063', '0.479', '0.155', '0.132', '0.431', '0.166', '0.485',
       '0.292', '0.265', '0.468', '0.086', '0.049', '0.069', '0.131',
       '0.228', '0.048', '0.109', '0.14', '0.158', '0.328', '0.057',
       '0.056', '0.099', '0.317', '0.601', '0.414', '0.204', '0.125',
       '0.088', '0.513', '0.299', '0.071', '0.102', '0.564', '0.032',
       '0.025', '0.018', '0.029', '0.54', '0.553', '0.139', '0.434',
       '0.483', '0.144', '0.061', '0.494', '0.474', '0.13', '0.067',
       '0.409', '0.013', '0.

In [20]:
emoji_df['Neut'].astype(float)
emoji_df['Pos'].astype(float)

1      0.468
2      0.790
3      0.693
4      0.729
5      0.343
       ...  
747    0.250
748    0.250
749    0.125
750    0.125
751    0.125
Name: Pos, Length: 751, dtype: float64

In [44]:
emoji_df['New_Neg'] = emoji_df['Neg']+(emoji_df['Neut']/2)
emoji_df['New_Pos'] = emoji_df['Pos']+(emoji_df['Neut']/2)

In [45]:
emoji_df.head()

,Char,Unicode,Occurrences,Position,Neg,Neut,Pos,Sentiment score,Unicode name,Unicode block,New_Neg,New_Pos
1,😂,0x1f602,14622,0.805,0.247,0.285,0.468,0.221,FACE WITH TEARS OF JOY,Emoticons,0.3895,0.6105
2,❤,0x2764,8050,0.747,0.044,0.166,0.790,0.746,HEAVY BLACK HEART,Dingbats,0.1270,0.8730
3,♥,0x2665,7144,0.754,0.035,0.272,0.693,0.657,BLACK HEART SUIT,Miscellaneous Symbols,0.1710,0.8290
4,😍,0x1f60d,6359,0.765,0.052,0.219,0.729,0.678,SMILING FACE WITH HEART-SHAPED EYES,Emoticons,0.1615,0.8385
5,😭,0x1f62d,5526,0.803,0.436,0.220,0.343,-0.093,LOUDLY CRYING FACE,Emoticons,0.5460,0.4530


In [46]:
emoji_df.to_csv("New_Emoji_Dataset.csv",index=False)